   ### Week 4 Assignment
   ### Capstone Project - The Battle of Neighborhoods
   |  Author: Mustafa Mahmoud

### Best location for African Resturant around newyork city/

## Import Libraries

In this section we import the libraries that will be required to process the data.


In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

import folium



Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Mustafa\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py38haa244fe_0         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.8             |           1_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0
  python_abi         conda-forge/win-64::python_abi-3.8-1_cp38

The following packa

ModuleNotFoundError: No module named 'folium'

In [3]:
! pip install folium
import folium


## Download and Explore Dataset


Download and Explore Dataset
Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.




In [47]:
#Download the data locally and import into json 
url='https://cocl.us/new_york_dataset'

with open('new_york_geojosn.json') as json_data:
    newyork_data = json.load(json_data)

#### Tranform  data into  *pandas* dataframe

In [48]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [49]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Use geopy library to get the latitude and longitude values of New York City.

In [50]:
#City of interest 
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [51]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.Borough.value_counts()

Manhattan    40
Name: Borough, dtype: int64

In [52]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Foursquare venues


In [53]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = '2I21WCHSLPW2VDRCBAWPD4TGGLJZX2WK1PFZGMJNMCDNDZN3'
CLIENT_SECRET = 'RUYCM0ZXLNZHBQV1JGVCBJ5PDCE1CANN511VXIXJU24W3FTB'
VERSION = '20201221'

In [54]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [55]:
neighborhoods.head(3)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806


In [56]:
#https://developer.foursquare.com/docs/resources/categories
#African resturant = 4bf58dd8d48988d1c8941735

#African  Resturant 
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_AFRICAN = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1c8941735')
newyork_venues_AFRICAN.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.715618,-73.994279,怡东楼,40.715741,-73.994271,African Restaurant
1,Chinatown,40.715618,-73.994279,Tolani,40.718599,-74.006181,African Restaurant
2,Chinatown,40.715618,-73.994279,Maman and Mimi,40.722834,-73.994449,Bistro
3,Hamilton Heights,40.823604,-73.949688,Tsion Cafe,40.826291,-73.943441,Ethiopian Restaurant
4,Hamilton Heights,40.823604,-73.949688,La Nomade Restaurant,40.825036,-73.940299,African Restaurant


In [57]:
print ("Number of African Resturant "+str(newyork_venues_AFRICAN.shape[0]))


Number of African Resturant 190


In [58]:
def showOnMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [59]:
map_newyork_AFRICAN = folium.Map(location=[latitude, longitude], zoom_start=10)
showOnMap(newyork_venues_AFRICAN, 'red', map_newyork_AFRICAN)

map_newyork_AFRICAN

In [60]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [61]:
manhattan_grouped = newyork_venues_AFRICAN.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(newyork_venues_AFRICAN['Venue Category'].unique())))
manhattan_grouped


There are 5 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,1,1,1,1,1,1
Carnegie Hill,2,2,2,2,2,2
Central Harlem,19,19,19,19,19,19
Chelsea,7,7,7,7,7,7
Chinatown,3,3,3,3,3,3
Civic Center,3,3,3,3,3,3
Clinton,7,7,7,7,7,7
East Harlem,3,3,3,3,3,3
East Village,7,7,7,7,7,7


In [62]:
manhattan_grouped.columns

Index(['Neighborhood Latitude', 'Neighborhood Longitude', 'Venue',
       'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

## 3. Analyze Each Neighborhood

In [63]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_AFRICAN[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_AFRICAN['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,African Restaurant,Bistro,Ethiopian Restaurant,Food Truck,French Restaurant
0,Chinatown,1,0,0,0,0
1,Chinatown,1,0,0,0,0
2,Chinatown,0,1,0,0,0
3,Hamilton Heights,0,0,1,0,0
4,Hamilton Heights,1,0,0,0,0


In [64]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,African Restaurant,Bistro,Ethiopian Restaurant,Food Truck,French Restaurant
0,Battery Park City,1.000000,0.000000,0.000000,0.0,0.000000
1,Carnegie Hill,1.000000,0.000000,0.000000,0.0,0.000000
2,Central Harlem,0.736842,0.000000,0.210526,0.0,0.052632
3,Chelsea,0.857143,0.000000,0.142857,0.0,0.000000
4,Chinatown,0.666667,0.333333,0.000000,0.0,0.000000
5,Civic Center,1.000000,0.000000,0.000000,0.0,0.000000
6,Clinton,0.571429,0.000000,0.428571,0.0,0.000000
7,East Harlem,1.000000,0.000000,0.000000,0.0,0.000000
8,East Village,0.571429,0.142857,0.285714,0.0,0.000000
9,Financial District,1.000000,0.000000,0.000000,0.0,0.000000


In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Battery Park City,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
1,Carnegie Hill,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
2,Central Harlem,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
3,Chelsea,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
4,Chinatown,African Restaurant,Bistro,French Restaurant,Food Truck,Ethiopian Restaurant


Cluster Neighborhoods


In [67]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 0, 3, 2, 4, 2, 3, 2])

In [68]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,NaN,NaN,NaN,NaN,NaN,NaN
1,Manhattan,Chinatown,40.715618,-73.994279,3.0,African Restaurant,Bistro,French Restaurant,Food Truck,Ethiopian Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,NaN,NaN,NaN,NaN,NaN,NaN
3,Manhattan,Inwood,40.867684,-73.921210,NaN,NaN,NaN,NaN,NaN,NaN
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4.0,Ethiopian Restaurant,African Restaurant,French Restaurant,Food Truck,Bistro


In [69]:
manhattan_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,NaN,NaN,NaN,NaN,NaN,NaN
1,Manhattan,Chinatown,40.715618,-73.994279,3.0,African Restaurant,Bistro,French Restaurant,Food Truck,Ethiopian Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,NaN,NaN,NaN,NaN,NaN,NaN
3,Manhattan,Inwood,40.867684,-73.921210,NaN,NaN,NaN,NaN,NaN,NaN
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4.0,Ethiopian Restaurant,African Restaurant,French Restaurant,Food Truck,Bistro
5,Manhattan,Manhattanville,40.816934,-73.957385,0.0,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
6,Manhattan,Central Harlem,40.815976,-73.943211,0.0,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
7,Manhattan,East Harlem,40.792249,-73.944182,2.0,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
8,Manhattan,Upper East Side,40.775639,-73.960508,2.0,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
9,Manhattan,Yorkville,40.775930,-73.947118,2.0,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro


In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [71]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Manhattanville,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
6,Central Harlem,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
15,Midtown,African Restaurant,Food Truck,Ethiopian Restaurant,French Restaurant,Bistro
17,Chelsea,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
25,Manhattan Valley,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
26,Morningside Heights,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
37,Stuyvesant Town,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
39,Hudson Yards,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro


In [72]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,Lincoln Square,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro,African Restaurant


In [73]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,East Harlem,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
8,Upper East Side,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
9,Yorkville,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
10,Lenox Hill,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
12,Upper West Side,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
16,Murray Hill,African Restaurant,Food Truck,French Restaurant,Ethiopian Restaurant,Bistro
21,Tribeca,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
27,Gramercy,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
28,Battery Park City,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro
29,Financial District,African Restaurant,French Restaurant,Food Truck,Ethiopian Restaurant,Bistro


In [74]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Chinatown,African Restaurant,Bistro,French Restaurant,Food Truck,Ethiopian Restaurant
18,Greenwich Village,African Restaurant,Ethiopian Restaurant,Bistro,French Restaurant,Food Truck
19,East Village,African Restaurant,Ethiopian Restaurant,Bistro,French Restaurant,Food Truck
20,Lower East Side,African Restaurant,Ethiopian Restaurant,Bistro,French Restaurant,Food Truck
22,Little Italy,African Restaurant,Ethiopian Restaurant,Bistro,French Restaurant,Food Truck
23,Soho,African Restaurant,Ethiopian Restaurant,Bistro,French Restaurant,Food Truck
31,Noho,African Restaurant,Ethiopian Restaurant,Bistro,French Restaurant,Food Truck


In [75]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Hamilton Heights,Ethiopian Restaurant,African Restaurant,French Restaurant,Food Truck,Bistro
14,Clinton,African Restaurant,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro
24,West Village,Ethiopian Restaurant,African Restaurant,French Restaurant,Food Truck,Bistro


East Harlem	 will be the most suitable place for the resturant, less compition 

In [76]:
result=manhattan_merged[manhattan_merged['Cluster Labels'] == 1]
result

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,Manhattan,Lincoln Square,40.773529,-73.985338,1.0,Ethiopian Restaurant,French Restaurant,Food Truck,Bistro,African Restaurant


In [77]:
markers_colors = []
for lat, lon, poi, cluster in zip(result['Latitude'], result['Longitude'], result['Neighborhood'], result['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color='ffffff',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters